<a href="https://colab.research.google.com/github/Sebastian-Frey/Timeseries-Forecasting-leveraging-LLMs/blob/main/paper_dtw_sem_neighbour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adding semantic and DTW neighbours to  *weekly_aggregated_by_week_domain_keyword.parquet*

## 1. Semantic neighbours


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# !pip -q install polars pyarrow numpy

from pathlib import Path
from datetime import date, timedelta
import numpy as np
import polars as pl

# ------------------ PATHS ------------------
PAPER = Path("/content/drive/MyDrive/colab_data/cleaned_cpu/paper")

IN_AGG   = PAPER / "weekly_aggregated_by_week_domain_keyword.parquet"
IDX_CSV  = PAPER / "keyword_domain_index_110.csv"
SIM_NPY  = PAPER / "keyword_similarity_matrix_110.npy"
OUT_FP   = PAPER / "weekly_aggregated_by_week_domain_keyword_sem_neighbours.parquet"

TOP_N = 20  # semantic_neighbour_1..5 (CPC of neighbor at t-1)

for p in (IN_AGG, IDX_CSV, SIM_NPY):
    if not p.exists():
        raise FileNotFoundError(p)

# ------------------ HELPERS ------------------
def iso_week_prev(wwyyyy: str) -> str:
    if wwyyyy is None:
        return None
    ww, yyyy = wwyyyy.split("-")
    d = date.fromisocalendar(int(yyyy), int(ww), 1)
    d_prev = d - timedelta(days=7)
    y, w, _ = d_prev.isocalendar()
    return f"{w:02d}-{y}"

# ------------------ 1) Rebuild keyword order used by SIM matrix ------------------
idx = pl.read_csv(IDX_CSV)

# In the SIM build we used: unique keywords, sorted alphabetically
keywords = (
    idx.select(pl.col("keyword").cast(pl.Utf8))
       .unique()
       .sort("keyword")
       .to_series()
       .to_list()
)
K = len(keywords)

sim = np.load(SIM_NPY, mmap_mode="r")
assert sim.shape == (K, K), "SIM matrix shape does not match reconstructed keyword list."

# ------------------ 2) Compute top-5 neighbor keywords for each keyword ------------------
neighbor_kw = np.empty((K, TOP_N), dtype=object)

for i in range(K):
    row = sim[i].astype(np.float32, copy=False)
    row[i] = -np.inf
    cand = np.argpartition(-row, TOP_N)[:TOP_N]
    cand = cand[np.argsort(-row[cand])]
    neighbor_kw[i, :] = [keywords[j] for j in cand]

neighbors_df = pl.DataFrame(
    {
        "keyword": keywords,
        **{f"nb_kw_{k}": neighbor_kw[:, k-1].tolist() for k in range(1, TOP_N+1)}
    }
)

# ------------------ 3) Load aggregated data + add week_prev + attach neighbor keywords ------------------
df = pl.read_parquet(IN_AGG)

df = (
    df.with_columns(
        pl.col("week").cast(pl.Utf8),
        pl.col("domain").cast(pl.Utf8, strict=False),
        pl.col("keyword").cast(pl.Utf8),
        pl.col("cpc_week").cast(pl.Float64, strict=False),
        pl.col("week").map_elements(iso_week_prev, return_dtype=pl.Utf8).alias("week_prev"),
    )
    .join(neighbors_df, on="keyword", how="left")
)

# ------------------ 4) Build GLOBAL CPC lookup by (week, keyword) ------------------
# If a (week, keyword) appears multiple times across domains, we aggregate to a single global CPC.
# Using ratio-of-sums is robust: global_cpc = sum(adcost_sum) / sum(adclicks_sum)
need_cost_clicks = ("adcost_sum" in df.columns) and ("adclicks_sum" in df.columns)

if need_cost_clicks:
    global_cpc = (
        df.select(["week", "keyword", "adcost_sum", "adclicks_sum"])
          .group_by(["week", "keyword"])
          .agg(
              pl.col("adcost_sum").sum().alias("adcost_sum_g"),
              pl.col("adclicks_sum").sum().alias("adclicks_sum_g"),
          )
          .with_columns(
              pl.when(pl.col("adclicks_sum_g") > 0)
                .then(pl.col("adcost_sum_g") / pl.col("adclicks_sum_g"))
                .otherwise(None)
                .alias("cpc_global")
          )
          .select(["week", "keyword", "cpc_global"])
    )
else:
    # fallback: average CPC across domains if you don't have sums
    global_cpc = (
        df.select(["week", "keyword", "cpc_week"])
          .group_by(["week", "keyword"])
          .agg(pl.col("cpc_week").mean().alias("cpc_global"))
    )

# ------------------ 5) Join neighbor GLOBAL CPCs from previous week ------------------
out = df
for k in range(1, TOP_N+1):
    out = (
        out.join(
            global_cpc.rename({"week": "week_prev", "keyword": f"nb_kw_{k}"}),
            on=["week_prev", f"nb_kw_{k}"],
            how="left",
        )
        .rename({"cpc_global": f"semantic_neighbour_{k}"})
    )

# Drop helper columns (keep original cols + semantic_neighbour_1..5)
drop_cols = ["week_prev"] + [f"nb_kw_{k}" for k in range(1, TOP_N+1)]
out = out.drop([c for c in drop_cols if c in out.columns])

# ------------------ 6) Write ------------------
out.write_parquet(str(OUT_FP), compression="zstd")
print(f"✅ Wrote: {OUT_FP}")
print("shape:", out.shape)
print(out.select(["week","domain","keyword","cpc_week"] + [f"semantic_neighbour_{k}" for k in range(1, TOP_N+1)]).head(8))


✅ Wrote: /content/drive/MyDrive/colab_data/cleaned_cpu/paper/weekly_aggregated_by_week_domain_keyword_sem_neighbours.parquet
shape: (199272, 46)
shape: (8, 24)
┌─────────┬────────┬────────────┬──────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ week    ┆ domain ┆ keyword    ┆ cpc_week ┆ … ┆ semantic_n ┆ semantic_n ┆ semantic_n ┆ semantic_n │
│ ---     ┆ ---    ┆ ---        ┆ ---      ┆   ┆ eighbour_1 ┆ eighbour_1 ┆ eighbour_1 ┆ eighbour_2 │
│ str     ┆ str    ┆ str        ┆ f64      ┆   ┆ 7          ┆ 8          ┆ 9          ┆ 0          │
│         ┆        ┆            ┆          ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│         ┆        ┆            ┆          ┆   ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞═════════╪════════╪════════════╪══════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 01-2021 ┆ avis   ┆ 3 month    ┆ 0.566589 ┆ … ┆ 0.934465   ┆ null       ┆ null       ┆ null       │
│         ┆        ┆ hire       

## 2. DTW Neighbours

In [5]:
!pip -q install polars pyarrow numpy tslearn tqdm

from pathlib import Path
from datetime import date, timedelta
import numpy as np
import polars as pl
from tslearn.metrics import cdist_dtw
from tqdm.auto import tqdm

# ------------------ CONFIG ------------------
PAPER = Path("/content/drive/MyDrive/colab_data/cleaned_cpu/paper")

# Global week-keyword CPC source (one row per week-keyword ideally)
IN_WK_KW = PAPER / "weekly_aggregated_by_week_domain_keyword_sem_neighbours.parquet"

# IMPORTANT: start from semantic-neighbour file so we DO NOT overwrite semantic_neighbour_*
IN_WDK_SEM = PAPER / "weekly_aggregated_by_week_domain_keyword_sem_neighbours.parquet"
# If you don't have it yet, fall back to base aggregated file:
IN_WDK_BASE = PAPER / "weekly_aggregated_by_week_domain_keyword.parquet"

OUT_FP = PAPER / "domain_sem_dtw.parquet"

HORIZON = 12
TOP_N = 20
SAKOE_RADIUS = 15
N_JOBS = 1          # keep low to avoid RAM spikes
CHUNK = 256         # distance chunk size

# ------------------ HELPERS ------------------
def parse_week_str(w: str):
    ww, yyyy = w.split("-")
    return int(yyyy), int(ww)

def iso_week_prev(wwyyyy: str) -> str:
    if wwyyyy is None:
        return None
    ww, yyyy = wwyyyy.split("-")
    d = date.fromisocalendar(int(yyyy), int(ww), 1)
    d_prev = d - timedelta(days=7)
    y, w, _ = d_prev.isocalendar()
    return f"{w:02d}-{y}"

# ------------------ 1) Load CPC by (week, keyword) and pick no-leak window ------------------
if not IN_WK_KW.exists():
    raise FileNotFoundError(IN_WK_KW)

wk_kw = (
    pl.read_parquet(IN_WK_KW, columns=["week", "keyword", "cpc_week", "adcost_sum", "adclicks_sum"])
      .with_columns(
          pl.col("week").cast(pl.Utf8),
          pl.col("keyword").cast(pl.Utf8),
          pl.col("cpc_week").cast(pl.Float64, strict=False),
          pl.col("adcost_sum").cast(pl.Float64, strict=False),
          pl.col("adclicks_sum").cast(pl.Float64, strict=False),
      )
)

weeks = wk_kw.select("week").unique().to_series().to_list()
weeks_sorted = sorted(weeks, key=parse_week_str)
cutoff_idx = max(0, len(weeks_sorted) - HORIZON)
weeks_train = weeks_sorted[:cutoff_idx]
if len(weeks_train) < 5:
    raise RuntimeError(f"Not enough training weeks after applying horizon={HORIZON}. Got {len(weeks_train)} weeks.")

print(f"Total weeks: {len(weeks_sorted)} | DTW uses: {len(weeks_train)} | Holdout: {HORIZON}")

# ------------------ 2) Build GLOBAL CPC lookup (ratio-of-sums) and pivot for DTW ------------------
# Global CPC per (week, keyword) across domains (ratio-of-sums is robust)
global_cpc = (
    wk_kw.group_by(["week", "keyword"])
         .agg(
             pl.col("adcost_sum").sum().alias("adcost_sum_g"),
             pl.col("adclicks_sum").sum().alias("adclicks_sum_g"),
         )
         .with_columns(
             pl.when(pl.col("adclicks_sum_g") > 0)
               .then(pl.col("adcost_sum_g") / pl.col("adclicks_sum_g"))
               .otherwise(None)
               .alias("cpc_global")
         )
         .select(["week", "keyword", "cpc_global"])
)

pivot = (
    global_cpc.filter(pl.col("week").is_in(weeks_train))
              .pivot(index="keyword", columns="week", values="cpc_global", aggregate_function="mean")
              .sort("keyword")
)

kw_list = pivot["keyword"].to_list()
week_cols = [c for c in pivot.columns if c != "keyword"]
week_cols = sorted(week_cols, key=parse_week_str)
pivot = pivot.select(["keyword"] + week_cols)

# ------------------ 3) CAUSAL IMPUTATION (no future) + z-normalize ------------------
vals = pivot.select(week_cols)
vals_ff = vals.select([pl.col(c).fill_null(strategy="forward") for c in week_cols])

X = vals_ff.to_numpy()
global_mean = np.nanmean(X)
if np.isnan(global_mean):
    global_mean = 0.0

# past-only expanding mean for leading NaNs
for i in range(X.shape[0]):
    s = 0.0
    n = 0
    for t in range(X.shape[1]):
        if not np.isnan(X[i, t]):
            s += X[i, t]
            n += 1
        else:
            if n > 0:
                X[i, t] = s / n

X = np.where(np.isnan(X), global_mean, X).astype(np.float32)

mu = X.mean(axis=1, keepdims=True)
sd = X.std(axis=1, keepdims=True)
sd[sd == 0] = 1.0
Xn = (X - mu) / sd

K = Xn.shape[0]
print("DTW input shape:", Xn.shape)

# ------------------ 4) Chunked DTW: compute only top-5 neighbors per keyword (NO full matrix) ------------------
neighbors = np.empty((K, TOP_N), dtype=object)

for i in tqdm(range(K), desc="DTW top-5 (chunked)"):
    best_d = np.full(TOP_N, np.inf, dtype=np.float32)
    best_j = np.full(TOP_N, -1, dtype=np.int32)

    xi = Xn[i:i+1]  # shape (1, T)

    for j0 in range(0, K, CHUNK):
        j1 = min(K, j0 + CHUNK)
        Xchunk = Xn[j0:j1]

        d = cdist_dtw(
            xi, Xchunk,
            global_constraint="sakoe_chiba",
            sakoe_chiba_radius=SAKOE_RADIUS,
            n_jobs=N_JOBS,
            verbose=False
        ).ravel().astype(np.float32)

        # exclude self if in chunk
        if j0 <= i < j1:
            d[i - j0] = np.inf

        # merge current chunk into best top-N
        cand_d = np.concatenate([best_d, d])
        cand_j = np.concatenate([best_j, np.arange(j0, j1, dtype=np.int32)])

        take = np.argpartition(cand_d, TOP_N)[:TOP_N]
        take = take[np.argsort(cand_d[take])]
        best_d = cand_d[take]
        best_j = cand_j[take]

    neighbors[i, :] = [kw_list[j] for j in best_j]

neighbors_df = pl.DataFrame(
    {"keyword": kw_list, **{f"dtw_nb_kw_{k}": neighbors[:, k-1].tolist() for k in range(1, TOP_N+1)}}
)

# ------------------ 5) Attach dtw_neighbour_1..5 CPC(t-1) to your (week,domain,keyword) table ------------------
if IN_WDK_SEM.exists():
    df = pl.read_parquet(IN_WDK_SEM)
    print("✅ Starting from semantic-neighbour file (will keep semantic_neighbour_* columns).")
elif IN_WDK_BASE.exists():
    df = pl.read_parquet(IN_WDK_BASE)
    print("⚠️ Semantic-neighbour file not found; starting from base aggregated file.")
else:
    raise FileNotFoundError("Neither semantic-neighbour nor base aggregated file exists.")

df = (
    df.with_columns(
        pl.col("week").cast(pl.Utf8),
        pl.col("domain").cast(pl.Utf8, strict=False),
        pl.col("keyword").cast(pl.Utf8),
        pl.col("week").map_elements(iso_week_prev, return_dtype=pl.Utf8).alias("week_prev"),
    )
    .join(neighbors_df, on="keyword", how="left")
)

out = df
for k in range(1, TOP_N+1):
    out = (
        out.join(
            global_cpc.rename({"week": "week_prev", "keyword": f"dtw_nb_kw_{k}", "cpc_global": "cpc_nb"}),
            on=["week_prev", f"dtw_nb_kw_{k}"],
            how="left",
        )
        .rename({"cpc_nb": f"dtw_neighbour_{k}"})
    )

# drop helper columns
drop_cols = ["week_prev"] + [f"dtw_nb_kw_{k}" for k in range(1, TOP_N+1)]
out = out.drop([c for c in drop_cols if c in out.columns])

# ------------------ 6) Write (overwrites same file, but now contains BOTH semantic_ and dtw_ columns) ------------------
out.write_parquet(str(OUT_FP), compression="zstd")
print(f"✅ Wrote: {OUT_FP}")
print("shape:", out.shape)

cols_sem = [c for c in out.columns if c.startswith("semantic_neighbour_")]
cols_dtw = [c for c in out.columns if c.startswith("dtw_neighbour_")]
print("semantic cols:", cols_sem)
print("dtw cols:", cols_dtw)
print(out.select(["week","domain","keyword"] + cols_sem[:min(2,len(cols_sem))] + cols_dtw).head(5))


Total weeks: 127 | DTW uses: 115 | Holdout: 12


/tmp/ipython-input-675703479.py:86: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  .pivot(index="keyword", columns="week", values="cpc_global", aggregate_function="mean")


DTW input shape: (1519, 115)


DTW top-5 (chunked):   0%|          | 0/1519 [00:00<?, ?it/s]

✅ Starting from semantic-neighbour file (will keep semantic_neighbour_* columns).
✅ Wrote: /content/drive/MyDrive/colab_data/cleaned_cpu/paper/domain_sem_dtw.parquet
shape: (199272, 66)
semantic cols: ['semantic_neighbour_1', 'semantic_neighbour_2', 'semantic_neighbour_3', 'semantic_neighbour_4', 'semantic_neighbour_5', 'semantic_neighbour_6', 'semantic_neighbour_7', 'semantic_neighbour_8', 'semantic_neighbour_9', 'semantic_neighbour_10', 'semantic_neighbour_11', 'semantic_neighbour_12', 'semantic_neighbour_13', 'semantic_neighbour_14', 'semantic_neighbour_15', 'semantic_neighbour_16', 'semantic_neighbour_17', 'semantic_neighbour_18', 'semantic_neighbour_19', 'semantic_neighbour_20']
dtw cols: ['dtw_neighbour_1', 'dtw_neighbour_2', 'dtw_neighbour_3', 'dtw_neighbour_4', 'dtw_neighbour_5', 'dtw_neighbour_6', 'dtw_neighbour_7', 'dtw_neighbour_8', 'dtw_neighbour_9', 'dtw_neighbour_10', 'dtw_neighbour_11', 'dtw_neighbour_12', 'dtw_neighbour_13', 'dtw_neighbour_14', 'dtw_neighbour_15', 'dtw_

In [7]:
print(out.columns)

['week', 'domain', 'keyword', 'impressions_sum', 'adcost_sum', 'adclicks_sum', 'cpc_week', 'avg_sim_top25_this_week', 'avg_sim_top25_last_week', 'n_sim_this_week', 'n_sim_last_week', 'dom_share_avis', 'dom_share_avisautonoleggio', 'dom_share_aviscarsales', 'dom_share_budget', 'dom_share_dollar', 'dom_share_economybookings', 'dom_share_hertz', 'dom_share_letsdrive', 'dom_share_sixt', 'dom_share_thrifty', 'n_dev_desktop', 'n_dev_mobile', 'n_dev_tablet', 'n_st_branded_search', 'n_st_generic_search', 'semantic_neighbour_1', 'semantic_neighbour_2', 'semantic_neighbour_3', 'semantic_neighbour_4', 'semantic_neighbour_5', 'semantic_neighbour_6', 'semantic_neighbour_7', 'semantic_neighbour_8', 'semantic_neighbour_9', 'semantic_neighbour_10', 'semantic_neighbour_11', 'semantic_neighbour_12', 'semantic_neighbour_13', 'semantic_neighbour_14', 'semantic_neighbour_15', 'semantic_neighbour_16', 'semantic_neighbour_17', 'semantic_neighbour_18', 'semantic_neighbour_19', 'semantic_neighbour_20', 'dtw_ne